# MMA Fighter Tracker - Interactive Demo

This notebook demonstrates the MMA Fighter Tracker system step by step, showing how to:
- Load and visualize video frames
- Run the tracker on MMA footage
- Analyze tracking results
- Visualize fighter trajectories

## Prerequisites

Make sure you have installed all dependencies:
```bash
pip install -r requirements.txt
```


In [ ]:
# Import required libraries
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
import subprocess
import warnings
warnings.filterwarnings('ignore')

# Set up plotting
plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully!")


## 1. Setup and Video Loading

First, let's check if we have the example video and set up our workspace.


In [ ]:
# Check for example video
example_video = "../example_data/UFC_20250803_Dricus_Du_Plessis_vs_Israel_Adesanya_FULL_FIGHT_UFC_319.f616 - chunk_17 [510-540]s.mp4"

if os.path.exists(example_video):
    print("Example video found!")
    
    # Get video properties
    cap = cv2.VideoCapture(example_video)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    duration = frame_count / fps
    
    print(f"Video Properties:")
    print(f"   Resolution: {width}x{height}")
    print(f"   FPS: {fps:.2f}")
    print(f"   Duration: {duration:.1f} seconds")
    print(f"   Total frames: {frame_count}")
    
    cap.release()
else:
    print("Example video not found!")
    print("Please ensure the example video is in the example_data/ directory")


## 2. Run the Tracker

Now let's run the MMA Fighter Tracker on our video. This will process the video and generate tracking results.


In [ ]:
# Set up output directory
output_dir = "../demo_results"
os.makedirs(output_dir, exist_ok=True)

# Run the tracker
print("Running MMA Fighter Tracker...")
print("This may take a few minutes depending on your hardware...")

cmd = [
    "python", "../src/tracker.py",
    "--video", example_video,
    "--outdir", output_dir,
    "--max-frames", "150",  # Process first 150 frames for demo
    "--stride", "5",        # Process every 5th frame
    "--conf", "0.5"         # Detection confidence threshold
]

try:
    result = subprocess.run(cmd, capture_output=True, text=True, timeout=300)
    
    if result.returncode == 0:
        print("Tracker completed successfully!")
        print("\nTracker Output:")
        print(result.stdout)
    else:
        print("Tracker failed!")
        print("Error:", result.stderr)
        
except subprocess.TimeoutExpired:
    print("Tracker timed out after 5 minutes")
except Exception as e:
    print(f"Error running tracker: {e}")


## 3. Analyze Results

Let's load the tracking results and analyze what the system detected.


In [ ]:
# Load tracking results
tracks_file = os.path.join(output_dir, "tracks.csv")

if os.path.exists(tracks_file):
    df = pd.read_csv(tracks_file)
    
    print("Tracking Results Summary:")
    print(f"   Total detections: {len(df)}")
    print(f"   Unique track IDs: {df['track_id'].nunique()}")
    print(f"   Frame range: {df['frame'].min()} - {df['frame'].max()}")
    
    # Track distribution
    track_counts = df['track_id'].value_counts().sort_index()
    print("\nTrack ID Distribution:")
    for track_id, count in track_counts.items():
        print(f"   Track {track_id}: {count} detections")
    
    # Frames with multiple fighters
    fighters_per_frame = df.groupby('frame')['track_id'].count()
    frames_with_2 = (fighters_per_frame == 2).sum()
    frames_with_1 = (fighters_per_frame == 1).sum()
    
    print(f"\nFighter Visibility:")
    print(f"   Frames with 2 fighters: {frames_with_2}")
    print(f"   Frames with 1 fighter: {frames_with_1}")
    
    # Display first few rows
    print("\nSample Data:")
    print(df.head(10))
    
else:
    print("No tracking results found!")
    print("Make sure the tracker ran successfully.")
